<img src="https://upload.wikimedia.org/wikipedia/commons/c/c7/HEIG-VD_Logo_96x29_RVB_ROUGE.png" alt="HEIG-VD Logo" width="250"/>

# Cours TAL - Mini - Projet
# Classification de dépêches d’agence avec NLTK

**Objectifs**

L’objectif de ce projet est de réaliser des expériences de *classification de documents* sous NLTK avec 
le corpus de dépêches Reuters. 

*  le corpus Reuters contient environ 10'000 dépêches datant des années 1980, et il est fourni avec NLTK comme expliqué dans le [livre NLTK, ch.2](http://www.nltk.org/book/ch02.html), §1.4.

In [2]:
import nltk
from nltk.corpus import reuters
nltk.downloader.Downloader().download('reuters') 
# à exécuter une seule fois pour télécharger les fichiers localement

[nltk_data] Downloading package reuters to C:\Users\Lenovo
[nltk_data]     T50s\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!


True

* Le code suivant illustre certaines de ses fonctionnalités en imprimant des informations de base pour la collection.

In [84]:
# List of documents
documents = reuters.fileids()
print(str(len(documents)) + " documents");
 
train_docs_id = list(filter(lambda doc: doc.startswith("train"),
documents));
print(str(len(train_docs_id)) + " total train documents");
 
test_docs_id = list(filter(lambda doc: doc.startswith("test"),documents));
print(str(len(test_docs_id)) + " total test documents");
 
# List of categories
categories = reuters.categories();
print(str(len(categories)) + " categories");
 
# Documents in a category
category_docs = reuters.fileids("interest");
 
# Words for a document
document_id = category_docs[0]
document_words = reuters.words(category_docs[0]);
print(document_words);
 
# Raw document
print(reuters.raw(document_id));

10788 documents
7769 total train documents
3019 total test documents
90 categories
['BUNDESBANK', 'ALLOCATES', '6', '.', '1', 'BILLION', ...]
BUNDESBANK ALLOCATES 6.1 BILLION MARKS IN TENDER
  The Bundesbank accepted bids for 6.1
  billion marks at today's tender for a 28-day securities
  repurchase pact at a fixed rate of 3.80 pct, a central bank
  spokesman said.
      Banks, which bid for a total 12.2 billion marks liquidity,
  will be credited with the funds allocated today and must buy
  back securities pledged on May 6.
      Some 14.9 billion marks will drain from the market today as
  an earlier pact expires, so the Bundesbank is effectively
  withdrawing a net 8.1 billion marks from the market with
  today's allocation.
      A Bundesbank spokesman said in answer to enquiries that the
  withdrawal of funds did not reflect a tightening of credit
  policy, but was to be seen in the context of plentiful
  liquidity in the banking system.
      Banks held an average 59.3 billion m

# Hyper-paramètres 

Avant d’appliquer les techniques traditionnelles d’apprentissage automatique, nous devons représenter et pondérer chaque document par rapport à l’ensemble des fonctionnalités textuelles. Nous allons appliquer les transformations suivantes :

* Minuscules le contenu d’origine
* Tokeniser le texte
* Stem chacun des jetons
* Pondérer chacune de ces caractéristiques, pour chaque document
* Normaliser la représentation

In [4]:
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
import re

from nltk.corpus import stopwords
stopWords = stopwords.words('english')
charfilter = re.compile('[a-zA-Z]+')

def simple_tokenizer(text):
    #tokenizing the words:
    #words = word_tokenize(text)
    #converting all the tokens to lower case:
    words = map(lambda word: word.lower(), text)
    #let's remove every stopwords
    words = [word for word in words if word not in stopWords]
    #stemming all the tokens
    tokens = (list(map(lambda token: PorterStemmer().stem(token), words)))
    ntokens = list(filter(lambda token : charfilter.match(token),tokens))
    return ntokens

In [5]:
import random
#permet de choisir de manière indépendant l'étiquête au choix
def selectRandom(names):
    return random.choice(names)

Pour la realisation de la premièere partie de se mini projet nous aveons décidé d'utiliser le maodule Scikit-learn .

* Scikit-learn est une bibliothèque d’apprentissage automatique open source pour python. Il fournit une variété d’algorithmes de régression, de classification et de clustering.
* l'idée fondamental est de comprendre comment fonction la methode suivante :sklearn.cross_validation.train_test_split(*tableaux, **options)[source]
* cette methode permet de diviser des tableaux ou des matrices en sous-ensembles aléatoires de train et de test.
* dans un premier temps il faut determiner le bon format pour les paramètre entrée de cette methode. 
* cette méthode retourne une  Liste contenant la répartition des entrées entre le train et le test.

pour les classifier nous avons choisi 3 classiffieur suivant:
* GAUSSIANNB
* LOGISTIC REGRESSION
* K-NEAREST NEIGHBORS





In [75]:
from nltk.corpus import stopwords, reuters
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split as tts
stop_words = stopwords.words("english")
 
# List of document ids 
def traitement(category):
    train_labels=[]

    train_docs = [reuters.raw(doc_id) for doc_id in documents]
    for doc_id in documents:
        if(doc_id in reuters.fileids(category)):
            train_labels.append("TRUE")
        else:
            train_labels.append("FALSE")

    # Tokenisation
    vectorizer = TfidfVectorizer(stop_words=stop_words, tokenizer=simple_tokenizer)

    vectorised_train_documents = vectorizer.fit_transform(train_docs).toarray()
    return tts(vectorised_train_documents, train_labels, test_size = 0.2, random_state = 42)


In [78]:
from sklearn.metrics import f1_score, precision_score, recall_score

# les scores de rappel, précision et f-mesure de chacun des trois 
classifieurs
def evaluation(y_test):
           # Evaluation
    precision = precision_score(y_test, predictions, average='micro')
    recall = recall_score(y_test, predictions, average='micro')
    f1 = f1_score(y_test, predictions, average='micro')
  
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}"
    .format(precision, recall, f1))
 
    precision = precision_score(y_test, predictions, average='macro')
    recall = recall_score(y_test, predictions, average='macro')
    f1 = f1_score(y_test, predictions, average='macro')
 
    print("Macro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))
    


# GaussianNB
Naive Bayes est une technique de classification statistique basée sur le théorème de Bayes.

dans le module Sklearn on l'utilise à travers les appelles suivant:
* classifier = GaussianNB()
* classifier.fit(X_train,  y_train)

In [85]:
from nltk.corpus import stopwords, reuters
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import f1_score, precision_score, recall_score
stop_words = stopwords.words("english")

#choix aleatoire de l'étiquette
names =["grain", "wheat", "corn"]     
category = selectRandom(names)

#traittement des données
X_train, X_test, y_train, y_test = traitement(category)

#classifier GaussianNB
classifier = GaussianNB()
classifier.fit(X_train,  y_train)
predictions = classifier.predict(X_test)

print("Name catégorie: "+str(category))
#prédiction avec la partie test de données
print("predictions: "+str(predictions))
evaluation(y_test)

 


C:\miniconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'e', 'f', 'g', 'h', 'j', 'l', 'n', 'p', 'r', 'u', 'v', 'w'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


Name catégorie: wheat
predictions: ['FALSE' 'FALSE' 'FALSE' ... 'FALSE' 'FALSE' 'FALSE']
Micro-average quality numbers
Precision: 0.7261, Recall: 0.7261, F1-measure: 0.7261
Macro-average quality numbers
Precision: 0.5314, Recall: 0.7377, F1-measure: 0.4848


# LOGISTIC REGRESSION
La régression logistique utilise une fonction logistique pour prédire une variable dépendante binaire.
pour l'utiliser Nous importons le package LogisticRegression comme suit :
* from sklearn.linear_model import LogisticRegression
puis appelé les méthodes suivantes:
* reg_log = LogisticRegression()
* reg_log.fit(X_train, y_train)

In [82]:
from nltk.corpus import stopwords, reuters
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import f1_score, precision_score, recall_score
stop_words = stopwords.words("english")

#choix aleatoire de l'étiquette
names =["money-fx", "interest", "money-supply"]     
category = selectRandom(names)

#traittement des données
X_train, X_test, y_train, y_test = traitement(category)

#classifier LOGISTIC REGRESSION
reg_log = LogisticRegression()
reg_log.fit(X_train, y_train)
y_pred = reg_log.predict(X_test)

print("Name catégorie: "+str(category))
#prédiction avec la partie test de données
print("predictions: "+str(predictions))
evaluation(y_test)

C:\miniconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'e', 'f', 'g', 'h', 'j', 'l', 'n', 'p', 'r', 'u', 'v', 'w'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


Name catégorie: money-fx
predictions: ['FALSE' 'FALSE' 'FALSE' ... 'FALSE' 'FALSE' 'FALSE']
Micro-average quality numbers
Precision: 0.7525, Recall: 0.7525, F1-measure: 0.7525
Macro-average quality numbers
Precision: 0.5205, Recall: 0.5626, F1-measure: 0.5011


# K-NEAREST NEIGHBORS
Les K voisins les plus proches utilisent des calculs de distance euclidienne où la prédiction est la moyenne des k voisins les plus proches.

pour l'utiliser Nous importons le package LogisticRegression comme suit :
* from sklearn.neighbors import KNeighborsClassifier
puis appelé les méthodes suivantes:
* reg_knn = KNeighborsClassifier()
* reg_knn.fit(X_train, y_train)

In [83]:
from nltk.corpus import stopwords, reuters
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import f1_score, precision_score, recall_score
stop_words = stopwords.words("english")

#choix aleatoire de l'étiquette
names =["crude","nat-gas", "gold"]     
category = selectRandom(names)

#traittement des données
X_train, X_test, y_train, y_test = traitement(category)

#classifier K-NEAREST NEIGHBORS
reg_knn = KNeighborsClassifier()
reg_knn.fit(X_train, y_train)
y_pred = reg_knn.predict(X_test)

print("Name catégorie: "+str(category))
#prédiction avec la partie test de données
print("predictions: "+str(predictions))
evaluation(y_test)

C:\miniconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'e', 'f', 'g', 'h', 'j', 'l', 'n', 'p', 'r', 'u', 'v', 'w'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


Name catégorie: crude
predictions: ['FALSE' 'FALSE' 'FALSE' ... 'FALSE' 'FALSE' 'FALSE']
Micro-average quality numbers
Precision: 0.7678, Recall: 0.7678, F1-measure: 0.7678
Macro-average quality numbers
Precision: 0.5326, Recall: 0.6229, F1-measure: 0.5144


* nous pouvons créer des classifieurs qui baliseront automatiquement les nouveaux documents avec des étiquettes de catégorie appropriées. Tout d’abord, nous construisons une liste de documents, étiquetés avec les catégories appropriées.

In [15]:
from nltk.corpus import reuters
import random

list_categorie = ["money-fx", "interest", "money-supply"]
document = [(list(reuters.words(reuterid)), category)
             for category in list_categorie
             for reuterid in reuters.fileids(category)]
random.shuffle(document)


# Classifieur multi-classe

In [25]:
list_categorie = ["money-fx", "grain", "crude"]

def get_custom_category(reuterid):
    for category in list_categorie:
        if reuterid in reuters.fileids(category):
            return category
    return 'other'

document = []
for reuterid in reuters.fileids():
    custom_category = get_custom_category(reuterid)
    document.append((list(reuters.words(reuterid)), custom_category))

random.shuffle(document)



In [26]:
featuresets = [(document_features(d), c) for (d,c) in document]
train_set, test_set = featuresets[100:], featuresets[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)


In [27]:
print(nltk.classify.accuracy(classifier, test_set))



0.62


In [28]:
import collections
from nltk.metrics import precision, recall, f_measure


ref_label_set = collections.defaultdict(set)
test_label_set = collections.defaultdict(set)

for i, (feats, label) in enumerate(test_set):
    ref_label_set[label].add(i)
    classification_label = classifier.classify(feats)
    test_label_set[classification_label].add(i)

print('Scores :')

for label in ref_label_set.keys():
    print(label)
    print( 'Precision :\t', precision(ref_label_set[label], test_label_set[label]) )
    print( 'Recall :\t', recall(ref_label_set[label], test_label_set[label]) )
    print( 'F-measure :\t', f_measure(ref_label_set[label], test_label_set[label]) )
    print('\n')

Scores :
other
Precision :	 1.0
Recall :	 0.5813953488372093
F-measure :	 0.7352941176470589


money-fx
Precision :	 0.2631578947368421
Recall :	 1.0
F-measure :	 0.41666666666666663


crude
Precision :	 0.2
Recall :	 0.8333333333333334
F-measure :	 0.3225806451612903


grain
Precision :	 0.3333333333333333
Recall :	 0.6666666666666666
F-measure :	 0.4444444444444444


